<a href="https://colab.research.google.com/github/Sarah0711/CUDA-Vision-Lab/blob/main/Basic_model_MSE_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [ ]:
import numpy as np
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch


#Define Variables

In [ ]:
n_epochs = 250
batch_size_train = 20
batch_size_test = 1000
learning_rate = 0.01

random_seed = 1
torch.manual_seed(random_seed)

#Load Data

In [ ]:
train_loader = torch.utils.data.DataLoader(
  dsets.MNIST('./data', train=True, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  dsets.MNIST('./data', train=False, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

0it [00:00, ?it/s]

9920512it [00:05, 1807948.09it/s]                             


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


32768it [00:00, 331782.61it/s]
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 5388578.67it/s]                           
8192it [00:00, 128219.94it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [ ]:
train_size = (len(train_loader.dataset))
test_size = (len(test_loader.dataset))
input_size = 784
hidden_size = 100
output_size = 10

#Defining the softmax function and derivatiev

In [ ]:
def softmax(z):
    z -= np.max(z,axis=1)[:,None]
    sm =  np.exp(z) / np.sum(np.exp(z), axis=1)[:,None]
    return sm

def softmax_grad(sm):
  grads = np.zeros(sm.shape)
  for i in range(grads.shape[0]):
      for j in range(sm.shape[0]):
        delta = 1 if i==j else 0
        grads[i] += np.multiply(sm[i],(delta-sm[j]))
  return grads


#Sigmoid activataion for hidden layer

In [ ]:
def sigmoid(x,deriv=False):
  if(deriv==True):
    return np.multiply(x,(1-x))
  return 1/(1+np.exp(-x)) 

#Mean Square Error

In [ ]:
def loss(e):
    """MSE loss function"""
    return np.mean((e)**2)

#Define weights and bias

In [ ]:
np.random.seed(random_seed)
w1 = np.random.randn(input_size,hidden_size)
b1 = np.zeros(hidden_size)
w2 = np.random.randn(hidden_size,output_size)
b2 = np.zeros(output_size)

#Calculate output and compare with label

In [ ]:
def evaluation():
  acc = 0.0
  for batch_idx, (test_data, test_target) in enumerate(test_loader):
      testx = test_data.reshape(batch_size_test,-1)
      testy = test_target.numpy()
      hidden_out = sigmoid(testx.numpy() @ w1 + b1)
      out = softmax(hidden_out @ w2 + b2 )  
      acc += np.mean(np.argmax(out,axis=1)==testy)

  acc /= batch_idx
  print('{0:0.09f}'.format(acc*100), "% of test examples classified correctly.")
  return 1.0-acc

#Training

In [ ]:

patience = 5
p = 0
lowest_e = 1.0
for epoch in range(n_epochs):

  number=0

  for batch_idx, (data, target) in enumerate(train_loader):
    trainx = data.reshape(batch_size_train,-1)
    trainy = np.zeros((batch_size_train,output_size))
    for i in range(batch_size_train):
      trainy[i,target[i]] = 1

  #feedforward
    hidden_out = sigmoid(trainx.numpy() @ w1 + b1)
    out = softmax(hidden_out @ w2 + b2 )

  #backpropagation errors
    error = trainy - out  
    delta_2 = np.multiply(error , softmax_grad(out))
    delta_w2 =  hidden_out.T @ delta_2
    delta_w2 *= (learning_rate /batch_size_train)
    
    delta_b2 = learning_rate/batch_size_train * delta_2.sum(axis=0) 
    
    delta_1 = np.multiply((delta_2 @ w2.T),sigmoid(hidden_out,True))
    delta_w1 =  trainx.numpy().T @ delta_1
    delta_w1 *= (learning_rate /batch_size_train)

    delta_b1 = learning_rate/batch_size_train * delta_1.sum(axis=0) 

    w2 -= delta_w2
    b2 -= delta_b2

    w1 -= delta_w1
    b1 -= delta_b1

  if(epoch%10==0):   
    e = evaluation()

    if e < lowest_e:
      lowest_e = e
      p = 0
      b_w2 = w2
      b_b2 = b2
      b_w1 = w1
      b_b1 = b1
    else:
      p += 1
    if p == patience:
      break
    

  if(epoch%1==0):   
    # print("error:",str(loss(error)))
    print('{0:0.09f}'.format(loss(error)), " mse of training examples.")


19.366666667 % of test examples classified correctly.
0.112289626  mse of training examples.
0.082976764  mse of training examples.
0.094728524  mse of training examples.
0.071332528  mse of training examples.
0.077693352  mse of training examples.
0.062202156  mse of training examples.
0.063133567  mse of training examples.
0.081338906  mse of training examples.
0.055626645  mse of training examples.
0.076210516  mse of training examples.
64.800000000 % of test examples classified correctly.
0.044538896  mse of training examples.
0.052850815  mse of training examples.
0.062739435  mse of training examples.
0.044706599  mse of training examples.
0.027355273  mse of training examples.
0.054464552  mse of training examples.
0.050546201  mse of training examples.
0.050942184  mse of training examples.
0.057128729  mse of training examples.
0.036706673  mse of training examples.
80.544444444 % of test examples classified correctly.
0.029212102  mse of training examples.
0.036726639  mse of

#Final Accuracy

In [ ]:

w2 = b_w2
b2 = b_b2
w1 = b_w1
b1 = b_b1

evaluation()

89.422222222 % of test examples classified correctly.


0.10577777777777753